In [ ]:
import itertools
import httpx
import pandas as pd

API_URL = "https://saisonmanager.de/api/v2"

leagues_url = API_URL + "/leagues.json"
league_url = API_URL + "/leagues/{league_id}.json"
standings_url = API_URL + "/leagues/{league_id}/table.json"
scorers_url = API_URL + "/leagues/{league_id}/scorer.json"
schedule_url = API_URL + "/leagues/{league_id}/schedule.json"
game_url = API_URL + "/games/{game_id}.json"

In [ ]:
leagues = pd.read_json(leagues_url)

# filter competitions by name (hopefully doesn't filter out something by accident)
leagues = leagues[
    (leagues.game_operation == "Floorball Deutschland")
    & (~leagues.name.str.contains("damen", case=False))
    & (~leagues.name.str.contains("junior", case=False))
    & (~leagues.name.str.contains("kleinfeld", case=False))
    & (~leagues.name.str.contains("KF", case=False))
    & (~leagues.name.str.contains("regio", case=False))
    & (~leagues.name.str.contains("pokal", case=False))
    & (~leagues.name.str.contains("rl", case=False))    
]

# sort
leagues.sort_values(by="season", ascending=False)

# export to csv
leagues_for_export=leagues[["id","name"]]
leagues_for_export.to_csv("v2_leagues.csv")

In [ ]:
# collect schedule data for the leagues; need to check that the URL can
# be reached beforehand because a few fail with a 500 Internal Server Error
schedules = {
    league_id: pd.read_json(schedule_url.format(league_id=league_id))
    for league_id in leagues.id
    if httpx.head(schedule_url.format(league_id=league_id)).is_success
}

In [ ]:
# create additional column containing league ids
league_ids = pd.Series(
    itertools.chain(
        *[
            itertools.repeat(league_id, len(schedule))
            for league_id, schedule in schedules.items()
        ]
    ),
    name="league_id",
)

# create matches dataframe from the combined schedules
matches = pd.concat(schedules.values(), ignore_index=True)

# unpack the JSON strings in the result column into a dataframe
results = pd.json_normalize(matches["result"])

# use nullable integer type
results["home_goals"] = results["home_goals"].astype("Int64")
results["guest_goals"] = results["guest_goals"].astype("Int64")

# remove JSON result column
matches = matches.drop(columns=["result"])

# combine the three dataframes into one with all the columns
matches = pd.concat([league_ids, results, matches.reset_index()], axis="columns")

# smaller dataframe
matches_small = matches[["league_id","game_id","date","home_team_name","guest_team_name","home_goals","guest_goals","overtime","forfait","ended"]][matches["ended"]==True]
matches_small['date']=pd.to_datetime(matches_small['date'],dayfirst=False,format="%Y-%m-%d")

# sort by date
matches_small_sorted=matches_small.sort_values(by='date')
matches_small_sorted

# export
matches_small_sorted.to_csv("v2_matches.csv")

In [ ]:
# Find individual teams
t1df=matches_small_sorted['home_team_name']
t1df.rename_axis('Team')
t2df=matches_small_sorted['guest_team_name']
teams=pd.concat([t1df,t2df],ignore_index=True)
indiv_teams = teams.unique()
indiv_teams.sort()
print(indiv_teams)

fout = open("v2_teams.csv","w")
for team in indiv_teams:
    fout.write(f"{team}\n")
fout.close()